In [1]:
import pandas as pd
import re

In [3]:
from zipfile import ZipFile
def extract_zip(path):
    with ZipFile(path,'r') as zip:
        zip.printdir()
        zip.extractall('/home/prajna/civicdatalab/scrapers/assam-tenders-data/data/RawData/unmapped_jan_sep2024')


In [4]:
# extract
PATH='/home/prajna/civicdatalab/scrapers/assam-tenders-data/data/RawData/unmapped_jan-sep2024.zip'
extract_zip(PATH)

File Name                                             Modified             Size
unmapped_jan-sep2024/                          2024-10-09 14:59:16            0
unmapped_jan-sep2024/final_2018_WPTBC_6506_1.csv 2024-09-30 14:14:38         6643
unmapped_jan-sep2024/final_2018_WPTBC_6507_1.csv 2024-09-30 14:14:34         7409
unmapped_jan-sep2024/final_2018_WPTBC_6508_1.csv 2024-09-30 14:14:28         6837
unmapped_jan-sep2024/final_2018_WPTBC_6509_1.csv 2024-09-30 14:14:24         7261
unmapped_jan-sep2024/final_2019_WPTBC_11277_1.csv 2024-09-30 14:14:18         7218
unmapped_jan-sep2024/final_2019_WPTBC_11286_1.csv 2024-09-30 14:14:12         9062
unmapped_jan-sep2024/final_2020_ASPIR_16050_1.csv 2024-09-30 14:14:08         7323
unmapped_jan-sep2024/final_2020_ASPIR_17272_1.csv 2024-09-30 14:14:04         8852
unmapped_jan-sep2024/final_2020_ASPIR_19080_1.csv 2024-09-30 14:13:58         7770
unmapped_jan-sep2024/final_2020_ASPIR_19634_1.csv 2024-09-30 14:13:52         7114
unmapped_jan-s

In [5]:
import pandas as pd
import re
import glob
import logging
import os
from datetime import datetime

In [6]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'csv_cleaner_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)

In [7]:
def clean_duplicate_columns(df, filename=""):
    try:
        logging.info(f"Starting column cleanup for {filename}")
        initial_cols = len(df.columns)
        
        columns_to_keep = []
        def get_base_name(col):
            return re.sub(r'\.\d+$', '', col)
        
        base_names = [get_base_name(col) for col in df.columns]
        
        def filter_columns(cols):
            columns_to_remove = ['Tender ID :', 'Tender Ref No :', 'Tender Title :']
            return [col for col in cols if col not in columns_to_remove]
        
        filtered_cols = filter_columns(df.columns)
        seen = set()
        
        for col, base in zip(filtered_cols, base_names):
            if base not in seen:
                columns_to_keep.append(col)
                seen.add(base)
        
        cleaned_df = df[columns_to_keep]
        final_cols = len(cleaned_df.columns)
        
        logging.info(f"Removed {initial_cols - final_cols} duplicate columns")
        return cleaned_df
        
    except Exception as e:
        logging.error(f"Error cleaning columns: {str(e)}")
        raise

In [10]:
def process_df_array(csv_input_path, csv_output_path):
    try:
        if not os.path.exists(csv_output_path):
            os.makedirs(csv_output_path)
            logging.info(f"Created output directory: {csv_output_path}")
        
        csv_filepaths = glob.glob(os.path.join(csv_input_path, "*.csv"))
        if not csv_filepaths:
            logging.warning(f"No CSV files found in {csv_input_path}")
            return
        
        logging.info(f"Found {len(csv_filepaths)} CSV files to process")
        
        for csv_filepath in csv_filepaths:
            try:
                filename = os.path.basename(csv_filepath)
                logging.info(f"Processing {filename}")
                
                df = pd.read_csv(csv_filepath)
                cleaned_df = clean_duplicate_columns(df, filename)
                
                output_file = os.path.join(csv_output_path, f"cleaned_{filename}")
                cleaned_df.to_csv(output_file, index=False)
                logging.info(f"Successfully saved cleaned file to {output_file}")
                
            except pd.errors.EmptyDataError:
                logging.error(f"Empty CSV file: {filename}")
                continue
            except pd.errors.ParserError:
                logging.error(f"Parser error in file: {filename}")
                continue
            except Exception as e:
                logging.error(f"Error processing {filename}: {str(e)}")
                continue
                
    except Exception as e:
        logging.error(f"Critical error in process_df_array: {str(e)}")
        raise

In [11]:
# invoke the main function
if __name__ == "__main__":
    try:
        input_path = "/home/prajna/civicdatalab/scrapers/assam-tenders-data/data/RawData/unmapped_jan_sep2024/unmapped_jan-sep2024"
        output_path = "/home/prajna/civicdatalab/scrapers/assam-tenders-data/data/RawData/unmapped_jan_sep2024_output_cleaned"
        process_df_array(input_path, output_path)
    except Exception as e:
        logging.critical(f"Application failed: {str(e)}")

2024-10-22 17:50:53,506 - INFO - Created output directory: /home/prajna/civicdatalab/scrapers/assam-tenders-data/data/RawData/unmapped_jan_sep2024_output_cleaned
2024-10-22 17:50:53,556 - INFO - Found 5495 CSV files to process
2024-10-22 17:50:53,558 - INFO - Processing final_2024_GMC_35325_12.csv
2024-10-22 17:50:53,929 - INFO - Starting column cleanup for final_2024_GMC_35325_12.csv
2024-10-22 17:50:54,021 - INFO - Removed 84 duplicate columns
2024-10-22 17:50:54,093 - INFO - Successfully saved cleaned file to /home/prajna/civicdatalab/scrapers/assam-tenders-data/data/RawData/unmapped_jan_sep2024_output_cleaned/cleaned_final_2024_GMC_35325_12.csv
2024-10-22 17:50:54,094 - INFO - Processing final_2024_BTC_37386_1.csv
2024-10-22 17:50:54,121 - INFO - Starting column cleanup for final_2024_BTC_37386_1.csv
2024-10-22 17:50:54,126 - INFO - Removed 84 duplicate columns
2024-10-22 17:50:54,132 - INFO - Successfully saved cleaned file to /home/prajna/civicdatalab/scrapers/assam-tenders-data/